In [7]:
%reset -f

In [26]:
from IPython.display import clear_output
import pandas as pd

In [9]:
class GolfSolitair:
    """Class for simulating a game of chess solitair"""

    BOARD_TEMPLATE = """
         ___
        / {} \\
       / {} {} \\
      / {} {} {} \\
     / {} {} {} {} \\
    / {} {} {} {} {} \\
    ⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻
    """

    # global constants
    ROWS = 5

    # maps targets relative to current position
    # {origin: {target: jumped}}
    BOARD_MAP = {
        0:  {3: 1, 5: 2},
        1:  {6: 3, 8: 4},
        2:  {7: 4, 9: 5},
        3:  {0: 1,  5: 4, 10: 6, 12: 7},
        4:  {11: 7, 13: 8},
        5:  {0: 2,  3: 4, 12: 8, 14: 9},
        6:  {1: 3,  8: 7},
        7:  {2: 4,  9: 8},
        8:  {1: 4,  6: 7},
        9:  {2: 5,  7: 8},
        10: {3: 6,  12: 11},
        11: {4: 7,  13: 12},
        12: {3: 7,  5: 8, 10: 11, 14: 13},
        13: {4: 8,  11: 12},
        14: {5: 9,  12: 13}
    }

    def __init__(self):
        self.reset()
        self.history = []

    def printBoard(self):
        """Prints the board to help display the game at any given state."""
        print(self.BOARD_TEMPLATE.format(*self._pegs), flush=True)

    def _isMoveValid(self, origin: int, target: int) -> bool:
        # origin is not empty.
        originValid = self._pegs[origin]
        # target is empty and reachable
        targetValid = (not self._pegs[target]) and (target in self.BOARD_MAP.get(origin).keys())
        # jumped target is not empty
        jumped = self.BOARD_MAP.get(origin).get(target)
        jumpedValid = jumped is not None and self._pegs[jumped]
        totalStatus = originValid and targetValid and jumpedValid
        return bool(totalStatus)

    def computeMoves(self) -> list[tuple[int, int]]:
        """Returns a list of tuples representing valid moves."""
        moves = []
        for origin, targets in self.BOARD_MAP.items():
            moves.extend([(origin, t) for t in targets.keys() if self._isMoveValid(origin, t)])
        return moves
        
    def performMove(self, origin: int, target: int):
        """Alters the pegs data struct according to the rules of game"""
        if self._isMoveValid(origin, target):
            jumped = self.BOARD_MAP.get(origin).get(target)
            self._pegs[origin] = 0
            self._pegs[jumped] = 0
            self._pegs[target] = 1
            self.history.append((origin, target))
        else:
            raise ValueError("Invalid origin/target")
        
    def performMoves(self, moves: list[tuple[int, int]]):
        [self.performMove(*m) for m in moves]
        
    def reset(self):
        # binary array with list of pegs
        self._pegs = [1]*len(self.BOARD_MAP)
        self._pegs[0] = 0


In [10]:
game = GolfSolitair()
game.printBoard()


         ___
        / 0 \
       / 1 1 \
      / 1 1 1 \
     / 1 1 1 1 \
    / 1 1 1 1 1 \
    ⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻
    


In [11]:
class Move:
    def __init__(self, parent, origin, target, children=None):
        self.parent = parent
        self.origin = origin
        self.target = target
        self.children = children

class GolfSolitairSimulator:

    def __init__(self):
        self.game = GolfSolitair()
        self.moves = []
        # constrain to the first move as other have is mirrors
        move = Move(None, *self.game.computeMoves()[0])
        self.moves.append(move)
        # for m in self.game.computeMoves():
            # move = Move(None, *m)
            # self.moves.append(move)
    
    def getTips(self):
        """Returns the moves furthers from the root"""
        children = []
        parents = self.moves[:]
        while parents:
            p: Move = parents.pop()
            if p.children is None:
                children.append(p)
            else:
                parents.extend(p.children)
        return children
    
    def getHistory(self, tip):
        """Gets the move history to get to the tip"""
        moves = [(tip.origin, tip.target)]
        while tip.parent is not None:
            tip = tip.parent
            moves.append((tip.origin, tip.target))
        return moves[::-1]

    def processLevel(self):
        tips = self.getTips()
        for t in tips:
            hist = self.getHistory(t)
            self.game.reset()
            self.game.performMoves(hist)
            children = self.game.computeMoves()
            for c in children:
                move = Move(t, *c)
                if t.children is None:
                    t.children = [move]
                else:
                    t.children.append(move)

        count = len(tips)
        return count

    def playGames(self):
        count, count_last = 0, -1
        level, results = 0, []
        while count != count_last:
            count_last = count
            count = self.processLevel()
            results.append((level, count))
            level += 1
        return results

In [12]:
# get a list of all the games
sim = GolfSolitairSimulator()
results = sim.playGames()
results

[(0, 1),
 (1, 4),
 (2, 21),
 (3, 111),
 (4, 537),
 (5, 2536),
 (6, 10766),
 (7, 38042),
 (8, 106433),
 (9, 207081),
 (10, 263493),
 (11, 280907),
 (12, 284315),
 (13, 284315)]

In [13]:
# grab the solutions from the solver
tips = sim.getTips()
wins = []
print(len(tips))

# cull out the results where more than one remains
for i, t in enumerate(tips):
    hist = sim.getHistory(t)
    sim.game.reset()
    sim.game.performMoves(hist)
    if sum(sim.game._pegs) == 1:
        wins.append(t)

len(wins)

284315


14880

In [14]:
# cull out equivelent solutions
histUniqueSet = set()
histUniqueList = []
for w in wins:
    hist = sim.getHistory(w)
    histTup = tuple(sorted(hist))
    if histTup not in histUniqueSet:
        histUniqueSet.add(histTup)
        histUniqueList.append(hist)

# here are the number of unique solutions
len(histUniqueList)

35

In [15]:
# 0.01% of games result in a unique win
len(histUniqueList)/results[-1][-1]

0.00012310289643529184

In [16]:
# 5.24 percent of games result in a non-unique win
len(wins)/results[-1][-1]

0.05233631711306122

In [28]:
# lets print off a list of the moves 
df = pd.DataFrame(histUniqueList)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(9, 7)","(6, 8)","(2, 9)","(3, 5)","(9, 2)","(0, 5)","(5, 12)","(13, 11)","(10, 12)"
1,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(9, 7)","(6, 1)","(5, 3)","(1, 6)","(6, 8)","(0, 5)","(5, 12)","(13, 11)","(10, 12)"
2,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(6, 1)","(5, 14)","(0, 3)","(3, 5)","(2, 9)","(14, 5)","(5, 12)","(13, 11)","(10, 12)"
3,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(6, 1)","(5, 12)","(13, 11)","(10, 12)","(1, 8)","(0, 5)","(9, 2)","(12, 5)","(5, 0)"
4,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(6, 1)","(5, 12)","(13, 11)","(10, 12)","(1, 8)","(0, 5)","(9, 2)","(12, 5)","(2, 9)"
5,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(6, 1)","(5, 12)","(13, 11)","(10, 12)","(0, 5)","(9, 2)","(2, 7)","(12, 3)","(3, 0)"
6,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(6, 1)","(5, 12)","(13, 11)","(10, 12)","(0, 5)","(9, 2)","(2, 7)","(12, 3)","(1, 6)"
7,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(6, 1)","(5, 3)","(13, 4)","(1, 6)","(10, 3)","(0, 5)","(9, 2)","(3, 5)","(5, 0)"
8,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(6, 1)","(5, 3)","(13, 4)","(1, 6)","(10, 3)","(0, 5)","(9, 2)","(3, 5)","(2, 9)"
9,"(3, 0)","(12, 3)","(14, 12)","(11, 13)","(6, 1)","(5, 3)","(13, 4)","(1, 6)","(10, 3)","(0, 5)","(9, 2)","(2, 7)","(3, 12)"


In [30]:
# interactive follow along game
def printGame(i, game: GolfSolitair):
    clear_output(wait=False)
    print(f"Playing game {i}", flush=True)
    game.printBoard()

i = 17
game = histUniqueList[i]
sim.game.reset()
printGame(i, sim.game)
for move in game:
    sim.game.performMove(*move)
    printGame(i, sim.game)
    input('Press enter to continue...')
# again = input('Play again? y/n').lower()
# if again != 'y':
#     break

Playing game 17

         ___
        / 0 \
       / 0 0 \
      / 0 0 0 \
     / 0 0 0 0 \
    / 0 0 1 0 0 \
    ⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻⁻
    
